# Machine Learning con BIG ML

## Tabla de contenidos:
* [Preparacion de datos](#datos-preparacion)
    * [Carga inicial de datos](#datos-carga-inicial)
    * [Preparación de los datos relevantes para BigML](#datos-preparacion-bigml)
    * [Volcado de datos en csv especifico](#datos-volcado)
* [Carga de datos en BigML](#bigml-carga)
    * [Carga de datos en la plataforma BigML](#bigml-carga-datos)
    * [Creación de un dataset](#bigml-creacion-dataset)
    * [Division del dataset](#bigml-division-dataset)
* [Modelo supervisado](#bigml-supervisado)
    * [Entrenando el modelo](#bigml-supervisado-entrenamiento)
    * [Resultados](#bigml-supervisado-resultados)
    * [Evaluación del modelo](#bigml-supervisado-evaluacion)
* [Modelo no supervisado](#bigml-nosupervisado)
    * [Detección de anomalías](#bigml-nosupervisado-anomalias)
* [Random Forest](#bigml-random-forest)
        

## Preparacion de datos <a class="anchor" name="datos-preparacion"/>

### Carga inicial de los datos <a class="anchor" name="datos-carga-inicial"/>

En primer lugar, tal y como se ha explicado en el notebook anterior<a href="./books.ipynb">(books.ipynb)</a>, se carga el en memoria el dataset de libros.

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/pedroortizgm/books-data-science/master/books_complete.csv")

### Preparación de los datos relevantes para BigML <a class="anchor" name="datos-preparacion-bigml"/>
Una vez que se ha creado el dataframe a partir del dataset se selecciona únicamente la información que se considera relevante para que en BigML a la hora de hacer un modelo de predicción indique cuando un libro va a tener buena acogida por parte de los lectores.

En concreto, los campos que se han considerado interesantes a cerca de cada libro son:
 + El título.
 + Los autores que lo han escrito.
 + La editorial.
 + Valoración media.
 + Número de valoraciones.
 + Fecha de publicación.
 + Generos a los que pertenece
 + Cuántas personas lo han leído.
 + Cuántas personas lo están leyendo.
 + Cuántas personas lo quieren leer.

### Volcado de datos en csv especifico <a class="anchor" name="datos-volcado"/>
Debido a que los campos "_genres_" y "_authors_" tienen valores multiples separados por comas, se guarda el CSV _"books-bigml.csv"_ con un nuevo separador (**|**).

Además se calcula una nueva columna *"good_book"* en la que se indica si un libro es bueno comprobando si valoración media es mayor o igual a la media de la valoración de todos los libros y además lo hayan valorado al menos 50 lectores. 

In [ ]:
books_average_mean = df.average_rating.mean()

def isGoodBook(book):
    return book.average_rating > books_average_mean

df = df.assign(good_book=isGoodBook)
df_bigml = df[["title", "authors", "publisher", "ratings_count", "original_publication_date", "genres", "read", "currently_reading", "to_read", "good_book"]]
df_bigml.to_csv(path_or_buf="books-bigml.csv", sep="|", index=False)

## Analisis de datos en BigML <a class="anchor" name="bigml-analisis"/>

### Carga de datos en la plataforma BigML <a class="anchor" name="bigml-carga-datos"/>
    
Para realizar la carga de datos se crea un nuevo proyecto llamado "Books" en el cual se carga el dataset que ha sido confeccionado con los campos mencionados anteriormente.

![carga-csv](images/carga-csv.png)



Como se ha guardado el CSV con un separador de tuberia, se configura la fuente de datos de forma adecuada, es decir, indicándole el tipo de separador de cada registro y el separador de los campos multivalor.

![source-configuration](images/source-configuration.png)



Se comprueba cómo está interpretando cada uno de los campos para verificar que el tipo de datos interpretado por BigML es el esperado. Podemos observar que el campo _"authors"_ y _"original_publication_date"_ tienen el tipo "text".

![source-configuration](images/dtype-analisis.png)


Para modifcarlo se puede hacer uso del siguiente menú de configuración:

![source-configuration](images/dtype-settings.png)


Una vez realizados los ajustes, se vuelve a comprobar en la siguiente pantalla que todo es correcto.

![source-configuration](images/dtype-final-analisis.png)

### Creación de un dataset <a class="anchor" name="bigml-creacion-dataset"/>

Una vez que la fuente de datos está bien configurada, mediante los botones indicados en la imagen se puede crear el dataset que permitirá construir los modelos de decisión (p.ej árboles de exploración de los datos).

![source-configuration](images/create-dataset.png)


### Division del dataset <a class="anchor" name="bigml-division-dataset"/>
Se parte el dataset en otros 2 dataset independientes.

+ Uno con el 80% del total de los datos que servirá para entrenar los modelos
+ y otro con el 20% restante de la información.

Éste último se usará para poder contrastar como de bueno es el modelo construido en las sesiones de entrenamiento.

![source-configuration](images/split-dataset.png)


## Modelo supervisado <a class="anchor" name="bigml-supervisado"/>

### Entrenando el modelo <a class="anchor" name="bigml-supervisado-entrenamiento"/>
Una vez han sido configurados los datasets llega el momento de entrenar el modelo con uno de ellos. Tal y como se ha comentando anteriormente, se empleará el que contiene el 80% de la información. 

Para ello se crea un modelo supervisado cuyo objetivo sea determinar si un libro será bueno.

![source-configuration](images/create-model.png)


### Resultados <a class="anchor" name="bigml-supervisado-resultados"/>
Después de procesar la información, BigML devuelve un gráfico con forma de árbol mediante el que se puede experimentar y buscar la rama más prometedora. En este caso, ha obtenido un nivel de precisión muy elevado. Se pueden ver a la derecha las condiciones que ha tenido en cuenta para predecir si se trata de un libro bueno.
 
![tree-prediction](images/tree-prediction.png)


### Evaluación del modelo <a class="anchor" name="bigml-supervisado-evaluacion"/>
Una vez que el modelo ha sido entrenado con el dataset de entrenamiento, ha llegado la hora de ponerlo a trabajar contra el otro dataset creado reservando el 20% de los datos originales de modo que se pueda contrastar cual es la precisión con la que trabaja el modelo y ver la cantidad de falsos positivos que nos da, así como la de falsos negativos.

En la siguiente pantalla se configuran los dataset que se van a usar para llevar a cabo la evaluación.

![model-evaluation-setup](images/model-evaluation-setup.png)

Haciendo clic en _"Evaluate"_ se obtienen los resultados:

![model-evaluation-conclusion](images/model-evaluation-conclusion.png)

Como se puede ver, lo primero que se establece es el valor _"Positive class"_ del gráfico a _"True"_. Con ello se indica que contraste los datos actuales con los de la predicción para saber cuantas veces hay un falso positivo (es decir, indica que un libro va a ser bueno cuando en la realidad no lo es) y cuantas un falso negativo (justo lo contrario, nos indica que el libro va a ser malo para los lectores, pero en realidad será bueno).

Se puede apreciar que hay 666 falsos positivos (naranjas en el gŕafico) y 79 falsos negativos (rojos). No es la mejor aproximación que se podría esperar del modelo, ya que para la industria literaria tener muchos falsos positivos seguramente signifique un gran desembolso económico en marketing y distribución de un producto que no va a tener el éxito esperado. 
Mientras que tener solamente 79 falsos negativos no es tan grave ya que significa que para esos libros no habrá tanta inversión para promover su venta y aún así obtendremos buenas ventas debido a su propia condición de ser bueno en realidad.

Es por ello que es conveniente explorar otros modelos antes de decidir que modelo usar para nuestras predicciones.

## Modelos no supervisados <a class="anchor" name="bigml-nosupervisado"/>

### Detección de anomalías <a class="anchor" name="bigml-nosupervisado-anomalias"/>
En la sección _"Datasets"_  del proyecto seleccionando en un menú contextual del dataset un nuevo tipo de dataset que generar, por ejemplo, anomalías. Al haberlo seleccionado se pasa a una pantalla que permitirá seleccionar sobre que campos se realizará la exploración de anomalías y construir con ello un dataset nuevo. Por defecto está configurado para detectar 10 anomalías o valores extraños en el dataset, por ejemplo, valoraciones medias negativas de los libros.

Se configura para que identifique 50 animalías y que genere el dataset libre de ellas.

![anomalies-detection](images/anomalies-detection.png)


El resultado es la siguiente pantalla en la que se muestra un resumen de las anomalías identificadas para que se añadan las que creamos oportunas antes de crear el dataset. Para el caso de uso que se valora, se seleccionan todas.

![create-50-anomalies-dataset](images/create-50-anomalies-dataset.png)

Si en la imagen anterior no se selecciona el botón a la izquierda de _"Create dataset"_, creará un dataset sólo con las anomalías detectadas. Este sería el resultado del nuevo dataset con solamente las anomalías

![top-50-anomalies-dataset](images/top-50-anomalies-dataset.png)

Sin embargo, lo que interesa es justo el caso contrario, de modo que se crea el dataset marcando esa opción. A partir de esos datos es posible crear de nuevo 2 dataset para entrenar otro modelo supervisado y ver si mejora respecto al caso anterior.

![training-without-anomalies](images/training-without-anomalies.png)


Se entrena de nuevo el modelo con el dataset que tiene el 80% de los datos libres de anomalias, y se obtiene el siguiente árbol de exploración
![tree-without-anomalies](images/tree-without-anomalies.png)

Por último, se evalua que tal rinde en este caso el modelo con la nueva situación para poder contrastarlo con la evaluación anterior y ver si ha mejorado el número de falsos positivos y falsos negativos.

![evaluation-without-anomalies](images/evaluation-without-anomalies.png)

Como se observa en la matriz de confusión, el número de falsos positivos y de falsos negativos ha aumentado ligeramente. Se concluye por tanto que el segundo modelo entrenado a partir de un dataset libre de anomalias no mejora la predición de si el libro será bueno o no.


## Modelo Random Forest <a class="anchor" name="bigml-random-forest"/>
Otro modo de construir un modelo competente es usando lo que se denomina Random Forest, o Emsemble en BigML. Consiste en entrenar "n" modelos simultáneamente para mejorar la precisión del modelo resultante final.

Para ello se configura hasta 15 modelos que serán entrenados con el 80% de los datos del dataset original. Con ello se evaluará de nuevo mediante la matriz de confusión si ha resultado de ayuda para el conjunto de datos introducido.

![random-forest-config](images/random-forest-config.png)

Se comprueban que campos tienen más peso en este tipo de modelos

![random-forest-field-importances](images/random-forest-field-importances.png)

Se evalua el modelo y se examina la matriz de confusión en la siguiente imagen comprobando que los resultados no mejoran a los obtenidos anteriormente, ya que se pasa a tener 689 falsos positivos. Sin embargo, el número de falsos negativos baja considerablemente.
Teniendo en cuenta que para el caso que ocupa (predecir si es buen libro) se debe minimizar los falsos positivos a costa de aumentar los falsos negativos, se considera mejor quedarse con el modelo inicial. 

![evaluate-random-forest](images/evaluate-random-forest.png)


<div style="width: 100%;">
    <div style="width: 33.33%; float: left; text-align: left;">
        <a href="./graficos.ipynb">Volver a Gráficos</a>
    </div>
    <div style="width: 33.33%; float: left; text-align: center">
        <span>David Julián Yela</span><br/>
        <span>Daniel Mateo Serrano</span><br/>
        <span>Pedro Ortiz García-Miguel</span>
    </div>
    <div style="width: 33.33%; float: left; text-align: right">
        &nbsp;
    </div>
</div>